<span style="font-size: 20px; font-weight: bold;">Installing Dependencies</span>

In [1]:
#Installing all the required packages
!pip install rasterio
!pip install matplotlib
!pip install plotly
!pip install pandas
!pip install ipywidgets
!pip install requests

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


Defaulting to user installation because normal site-packages is not writeable


<span style="font-size: 20px; font-weight: bold;">Importing Libraries</span>

In [2]:
import rasterio
from rasterio.plot import show
import plotly.io as pio
import plotly.express as px
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
from IPython.display import clear_output
import requests
import json

<span style="font-size: 20px; font-weight: bold;">This Function visualizes COG (Cloud Optimized Geotiff Images) on Mapbox</span>

In [3]:
#this function visualized any given COG inside of a mapbox view
def visualize_COG(layer_url, zoom=None):
    pio.renderers.default = 'notebook'
    # pio.renderers.default = 'jupyterlab'
    # pio.renderers.default = 'colab'

    # seed value, for mapbox to load in colab
    df = pd.DataFrame([[1001, 5.3],[1001, 5.3]])
    df.columns = ["flips", "unemp"]
    fig = px.choropleth_mapbox(
                                df, 
                                color='unemp',
                                color_continuous_scale="Viridis",
                                range_color=(0, 12),
                                mapbox_style="carto-positron",
                                zoom=4, center = {"lat": 33.543682, "lon": -86.779633},
                                opacity=0.5,
                              )
    fig.update_layout(
        mapbox_layers=[
            {
                "sourcetype": "raster",
                "source": [layer_url]
            }
          ])
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    fig.update_layout(mapbox_style="stamen-terrain")
    fig.layout.mapbox.zoom = 1
    fig.show()

<span style="font-size: 20px; font-weight: bold;">Constant Variables and Dataset Mappers</span>

In [4]:
#constants
DATASETS = ['TRMM_LIS_FULL','TRMM_LIS_SEASONAL','TRMM_LIS_MONTHLY', 'TRMM_LIS_DIURNAL', 'TRMM_LIS_DAILY',
            'OTD_FULL','OTD_MONTHLY','OTD_DIURNAL','OTD_DAILY', 'ISSLIS', 'NALMA', 'HS3']
BASE_URL = 'https://wug8w3fg42.execute-api.us-west-2.amazonaws.com/development/singleband/'

#Datasets mapper

#season_data
seasons = {'Spring':'2013_03_01','Summer':'2013_07_01','Autumn':'2013_10_01','Winter':'2013_12_01'}

#month_data
months = {}
for i in range(9):
    months[f'{i+1}'] = f"20130{i+1}"
months['10'] = '201310'
months['11'] = '201311'
months['12'] = '201312'

#Diurnal_data
diurnal = {}
hour = 1
for i in range(9):
    diurnal[f'Hour {hour}'] = f'2013_0{i+1}_01'
    hour+=1
    diurnal[f'Hour {hour}'] = f'2013_0{i+1}_15'
    hour+=1
for i in range(3):
    diurnal[f'Hour {hour}'] = f'2013_{i+10}_01'
    hour+=1
    diurnal[f'Hour {hour}'] = f'2013_{i+10}_15'
    hour+=1

#Daily data
daily = {}
calendar = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
count = 1
day = 0
for x in range(12):
    month = '00'
    if(x+1 <= 9):
        month = f'0{x+1}'
    else:
        month = f'{x+1}'

    for i in range(calendar[x]): 
        if i+1 <= 9:
            daily[f"Day {count}"] = f"2013_{month}_0{i+1}"
        else:
            daily[f"Day {count}"] = f'2013_{month}_{i+1}'
        count+=1
        
#returns all the key in a dictionary
def get_key(data):
    key = [key for key, value in data.items()]
    return key

<span style="font-size: 20px; font-weight: bold;">These methods help with Visualizing DropDowns for Inputs</span>

In [5]:
#input handler for dropdown
def handle_dataset_input(options, selected_option, id, metadata={}, color_map=None):  
    metadata = clear_metadata(id, options, selected_option, metadata)
    if metadata['name']['selected_option'] == DATASETS[0]:
        handle_trmm_lis_full(metadata, color_map=color_map)
    elif metadata['name']['selected_option'] == DATASETS[1]:
        handle_trmm_lis_seasonal(metadata)
    elif metadata['name']['selected_option'] == DATASETS[2]:
        handle_trmm_lis_monthly(metadata, color_map=color_map)
    elif metadata['name']['selected_option'] == DATASETS[3]:
        handle_trmm_lis_diurnal(metadata)
    elif metadata['name']['selected_option'] == DATASETS[4]:
        handle_trmm_lis_daily(metadata, color_map=color_map)
    elif metadata['name']['selected_option'] == DATASETS[5]:
        handle_otd_full(metadata, color_map=color_map)
    elif metadata['name']['selected_option'] == DATASETS[6]:
        handle_otd_monthly(metadata, color_map=color_map)
    elif metadata['name']['selected_option'] == DATASETS[7]:
        handle_otd_diurnal(metadata, color_map=color_map)
    elif metadata['name']['selected_option'] == DATASETS[8]:
        handle_otd_daily(metadata, color_map=color_map)
    elif metadata['name']['selected_option'] == DATASETS[9]:
        handle_isslis(metadata, color_map=color_map)
    elif metadata['name']['selected_option'] == DATASETS[10]:
        handle_nalma(metadata)
    elif metadata['name']['selected_option'] == DATASETS[11]:
        handle_hs3(metadata, color_map=color_map)
        
def clear_metadata(id, options, selected_option, metadata):
    if id == 'name':
        metadata = {}
        metadata['name'] = {}
        if metadata.get('arg2') != None:
            metadata['arg2'] = {}
        if metadata.get('arg3') != None:
            metadata['arg3'] = {}
        if metadata.get('arg4') != None:
            metadata['arg4'] = {}
        if metadata.get('arg5') != None:
            metadata['arg5'] = {}
        metadata['name']['options'] = options
        metadata['name']['selected_option'] = selected_option
    if id == 'arg2':
        metadata['arg2'] = {}
        if metadata.get('arg3') != None:
            metadata['arg3'] = None
        if metadata.get('arg4') != None:
            metadata['arg4'] = None
        if metadata.get('arg5') != None:
            metadata['arg5'] = None
        metadata['arg2']['options'] = options
        metadata['arg2']['selected_option'] = selected_option
    if id == 'arg3':
        metadata['arg3'] = {}
        if metadata.get('arg4') != None:
            metadata['arg4'] = None
        if metadata.get('arg5') != None:
            metadata['arg5'] = None
        metadata['arg3']['options'] = options
        metadata['arg3']['selected_option'] = selected_option
    if id == 'arg4':
        metadata['arg4'] = {}
        if metadata.get('arg5') != None:
            metadata['arg5'] = None
        metadata['arg4']['options'] = options
        metadata['arg4']['selected_option'] = selected_option
    if id == 'arg5':
        metadata['arg5'] = {}
        metadata['arg5'] = {}
        metadata['arg5']['options'] = options
        metadata['arg5']['selected_option'] = selected_option
    return metadata

<span style="font-size: 20px; font-weight: bold;">Input handlers for all the datasets</span>

In [6]:
#handlers for different datasets

################################################Handlers for TRMM-LIS dataset################################################
def handle_trmm_lis_full(metadata={}, color_map=None):
    clear_cell_and_render(metadata)
    visualize_COG(BASE_URL+"VHRFC/201301/LIS/{z}/{x}/{y}.png?colormap=terrain&stretch_range=[0.325,23.426]")
def handle_trmm_lis_seasonal(metadata={}, color_map=None):
    clear_cell_and_render(metadata) 
    #name is already selected, ask for which season now
    if metadata.get('arg2') != None:
        url = f"VHRSC/{seasons[metadata['arg2']['selected_option']]}"+"/LIS/{z}/{x}/{y}.png?colormap=terrain&stretch_range=[0.00010455249866936356,0.06766455620527267]"
        visualize_COG(BASE_URL+url)
    else:
        print("Select season: ", end="")
        drop_down(get_key(seasons), id='arg2', metadata=metadata)
def handle_trmm_lis_monthly(metadata={}, color_map=None):
    clear_cell_and_render(metadata) 
    #name is already selected, ask for which season now
    if metadata.get('arg2') != None:
        url = f"VHRMC/{months[metadata['arg2']['selected_option']]}"+"/LIS/{z}/{x}/{y}.png?colormap=terrain&stretch_range=[0.00010455249866936356,0.06766455620527267]"
        visualize_COG(BASE_URL+url)
    else:
        print("Select month: ", end="")
        drop_down(get_key(months), id='arg2', metadata=metadata)
def handle_trmm_lis_diurnal(metadata={}, color_map=None):
    clear_cell_and_render(metadata) 
    #name is already selected, ask for which season now
    if metadata.get('arg2') != None:
        url = f"VHRDC/{diurnal[metadata['arg2']['selected_option']]}"+"/LIS/{z}/{x}/{y}.png?colormap=terrain&stretch_range=[0.00010455249866936356,0.06766455620527267]"
        visualize_COG(BASE_URL+url)
    else:
        print("Select Time: ", end="")
        drop_down(get_key(diurnal), id='arg2', metadata=metadata)
def handle_trmm_lis_daily(metadata={}, color_map=None):
    clear_cell_and_render(metadata) 
    #name is already selected, ask for which season now
    if metadata.get('arg2') != None:
        url = f"VHRAC/{daily[metadata['arg2']['selected_option']]}"+"/LIS/{z}/{x}/{y}.png?colormap=terrain&stretch_range=[0.00010455249866936356,0.06766455620527267]"
        visualize_COG(BASE_URL+url)
    else:
        print("Select day: ", end="")
        drop_down(get_key(daily), id='arg2', metadata=metadata)
        
##################################################Handlers for OTD dataset################################################
def handle_otd_full(metadata={}, color_map=None):
    clear_cell_and_render(metadata)
    visualize_COG(BASE_URL+"HRFC/201301/OTD/{z}/{x}/{y}.png?colormap=terrain&stretch_range=[0.325,55.426]")
def handle_otd_monthly(metadata={}, color_map=None):
    clear_cell_and_render(metadata) 
    #name is already selected, ask for which season now
    if metadata.get('arg2') != None:
        url = f"HRMC/{months[metadata['arg2']['selected_option']]}"+"/OTD/{z}/{x}/{y}.png?colormap=terrain&stretch_range=[0,0.2]"
        visualize_COG(BASE_URL+url)
    else:
        print("Select month: ", end="")
        drop_down(get_key(months), id='arg2', metadata=metadata)
def handle_otd_diurnal(metadata={}, color_map=None):
    clear_cell_and_render(metadata) 
    #name is already selected, ask for which season now
    if metadata.get('arg2') != None:
        url = f"LRDC/{diurnal[metadata['arg2']['selected_option']]}"+"/OTD/{z}/{x}/{y}.png?colormap=terrain&stretch_range=[0,0.002]"
        visualize_COG(BASE_URL+url)
    else:
        print("Select Time: ", end="")
        drop_down(get_key(diurnal), id='arg2', metadata=metadata)
def handle_otd_daily(metadata={}, color_map=None):
    clear_cell_and_render(metadata) 
    #name is already selected, ask for which season now
    if metadata.get('arg2') != None:
        url = f"HRAC/{daily[metadata['arg2']['selected_option']]}"+"/OTD/{z}/{x}/{y}.png?colormap=terrain&stretch_range=[0.00010455249866936356,0.06766455620527267]"
        visualize_COG(BASE_URL+url)
    else:
        print("Select day: ", end="")
        drop_down(get_key(daily), id='arg2', metadata=metadata)
###################################################Handlers for isslis, nalma and hs3######################################
base_url = "https://s23k5d19rl.execute-api.us-west-2.amazonaws.com/"
def handle_isslis(metadata={}, color_map=None):
    clear_cell_and_render(metadata)
    dataset_name = "isslis"
    if metadata.get('arg2') == None:
        print("Select year: ", end="")
        response = requests.get(f"{base_url}arg1?name={dataset_name}")
        response = json.loads(response.text)
        drop_down(response, id='arg2', metadata=metadata)
    else:
        if metadata.get('arg3') == None:
            print("Select month: ", end="")
            response = requests.get(f"{base_url}arg2?name={dataset_name}&year={metadata['arg2']['selected_option']}")
            response = json.loads(response.text)
            drop_down(response, id='arg3', metadata=metadata)
        else:
            if metadata.get('arg4') == None:
                print("Select day: ", end="")
                response = requests.get(f"{base_url}arg3?name={dataset_name}&year={metadata['arg2']['selected_option']}&month={metadata['arg3']['selected_option']}")
                response = json.loads(response.text)
                drop_down(response, id='arg4', metadata=metadata)
            else:
                if metadata.get('arg5') == None:
                    print("Select Time/Instrumnet: ", end="")
                    response = requests.get(f"{base_url}arg4?name={dataset_name}&year={metadata['arg2']['selected_option']}&month={metadata['arg3']['selected_option']}&day={metadata['arg4']['selected_option']}")
                    response = json.loads(response.text)
                    drop_down(response, id='arg5', metadata=metadata)
                else:
                    year = metadata['arg2']['selected_option']
                    month = metadata['arg3']['selected_option']
                    day = metadata['arg4']['selected_option']
                    arg4 = metadata['arg5']['selected_option']
                    url = f"ISS_LIS/{year}{month}{day}/{arg4}"+"/{z}/{x}/{y}.png?colormap=terrain&stretch_range=[0,0]"
                    visualize_COG(BASE_URL+url)
    
def handle_nalma(metadata={}, color_map=None):
    clear_cell_and_render(metadata)
    dataset_name = "nalma"
    if metadata.get('arg2') == None:
        print("Select year: ", end="")
        response = requests.get(f"{base_url}arg1?name={dataset_name}")
        response = json.loads(response.text)
        drop_down(response, id='arg2', metadata=metadata)
    else:
        if metadata.get('arg3') == None:
            print("Select month: ", end="")
            response = requests.get(f"{base_url}arg2?name={dataset_name}&year={metadata['arg2']['selected_option']}")
            response = json.loads(response.text)
            drop_down(response, id='arg3', metadata=metadata)
        else:
            if metadata.get('arg4') == None:
                print("Select day: ", end="")
                response = requests.get(f"{base_url}arg3?name={dataset_name}&year={metadata['arg2']['selected_option']}&month={metadata['arg3']['selected_option']}")
                response = json.loads(response.text)
                drop_down(response, id='arg4', metadata=metadata)
            else:
                if metadata.get('arg5') == None:
                    print("Select Time/Instrumnet: ", end="")
                    response = requests.get(f"{base_url}arg4?name={dataset_name}&year={metadata['arg2']['selected_option']}&month={metadata['arg3']['selected_option']}&day={metadata['arg4']['selected_option']}")
                    response = json.loads(response.text)
                    drop_down(response, id='arg5', metadata=metadata)
                else:
                    year = metadata['arg2']['selected_option']
                    month = metadata['arg3']['selected_option']
                    day = metadata['arg4']['selected_option']
                    arg4 = metadata['arg5']['selected_option']
                    arg4 = arg4.split("_")
                    url = f"NALMA_{arg4[0]}/{year}{month}{day}/{arg4[1]}"+"/{z}/{x}/{y}.png?colormap=terrain&stretch_range=[0,1]"
                    visualize_COG(BASE_URL+url)
def handle_hs3(metadata={}, color_map=None):
    clear_cell_and_render(metadata)
    dataset_name = "hs3"
    if metadata.get('arg2') == None:
        print("Select year: ", end="")
        response = requests.get(f"{base_url}arg1?name={dataset_name}")
        response = json.loads(response.text)
        drop_down(response, id='arg2', metadata=metadata)
    else:
        if metadata.get('arg3') == None:
            print("Select month: ", end="")
            response = requests.get(f"{base_url}arg2?name={dataset_name}&year={metadata['arg2']['selected_option']}")
            response = json.loads(response.text)
            drop_down(response, id='arg3', metadata=metadata)
        else:
            if metadata.get('arg4') == None:
                print("Select day: ", end="")
                response = requests.get(f"{base_url}arg3?name={dataset_name}&year={metadata['arg2']['selected_option']}&month={metadata['arg3']['selected_option']}")
                response = json.loads(response.text)
                drop_down(response, id='arg4', metadata=metadata)
            else:
                if metadata.get('arg5') == None:
                    print("Select Time/Instrumnet: ", end="")
                    response = requests.get(f"{base_url}arg4?name={dataset_name}&year={metadata['arg2']['selected_option']}&month={metadata['arg3']['selected_option']}&day={metadata['arg4']['selected_option']}")
                    response = json.loads(response.text)
                    drop_down(response, id='arg5', metadata=metadata)
                else:
                    year = metadata['arg2']['selected_option']
                    month = metadata['arg3']['selected_option']
                    day = metadata['arg4']['selected_option']
                    arg4 = metadata['arg5']['selected_option']
                    url = f"HS3/{year}{month}{day}/{arg4}"+"/{z}/{x}/{y}.png?colormap=terrain&stretch_range=[0,0]"
                    visualize_COG(BASE_URL+url)

<span style="font-size: 20px; font-weight: bold;">Give inputs to dropdowns for dataset Visualization</span>

In [18]:
#renders dropdowns 
def drop_down(options=DATASETS, value=None , _id='name', metadata={}, color_map="terrain"):

    if _id == 'name':
        print("Select a dataset from the options below: ", end="")

    output = widgets.Output()
    dropdown = None
    # Create the dropdown widget
    if value != None:
        dropdown = widgets.Dropdown(options=options, description='', value=value)
    else:
        dropdown = widgets.Dropdown(options=options, description='', value=None)
    # Define a function to handle the dropdown's value change
    def handle_dropdown_change(change):
        with output:
            clear_output()
            selected_option = change.new
            handle_dataset_input(options, selected_option, _id, metadata=metadata, color_map=color_map)
    # Attach the function to the dropdown's value change event
    dropdown.observe(handle_dropdown_change, names='value')
    # Display the dropdown widget
    display(dropdown, output)

    #clears a cell output and re-renders the dropdowns based on metadata
def clear_cell_and_render(metadata={}):
    pass

In [19]:
colormaps = ['terrain', 'winter', 'ylorrd', 'twilight_shifted', 'cmrmap', 'jet', 'nipy_spectral', 'ylgnbu', 'twilight']

In [20]:
color_map = colormaps[0]

In [17]:
print("Select a dataset from the options below: ", end="")
drop_down(color_map=color_map)

Select a dataset from the options below: Select a dataset from the options below: 

Dropdown(options=('TRMM_LIS_FULL', 'TRMM_LIS_SEASONAL', 'TRMM_LIS_MONTHLY', 'TRMM_LIS_DIURNAL', 'TRMM_LIS_DAIL…

Output()